<a href="https://colab.research.google.com/github/DominionAkinrotimi/HAMOYE-DS-INTERNSHIP-PROJECTS/blob/main/STAGE_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [43]:
df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [33]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [44]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(0, inplace=True)

In [59]:
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

In [61]:
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                        'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [62]:
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(train_df[numerical_features])

scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_features)

In [80]:
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical = encoder.fit_transform(train_df[categorical_features])

encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

In [64]:
X_train = pd.concat([scaled_numerical_df.reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)
y_train = train_df['Churn']

# Applyingg the same transformations to the test data
scaled_numerical_test = scaler.transform(test_df[numerical_features])
scaled_numerical_test_df = pd.DataFrame(scaled_numerical_test, columns=numerical_features)

encoded_categorical_test = encoder.transform(test_df[categorical_features])
encoded_categorical_test_df = pd.DataFrame(encoded_categorical_test, columns=encoder.get_feature_names_out(categorical_features))

X_test = pd.concat([scaled_numerical_test_df.reset_index(drop=True), encoded_categorical_test_df.reset_index(drop=True)], axis=1)
y_test = test_df['Churn']

In [66]:
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')

In [81]:
# Initializing models
rf = RandomForestClassifier(random_state=1)
et = ExtraTreesClassifier(random_state=1)
xgb = XGBClassifier(random_state=1, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(random_state=1)

# To train the models
rf.fit(X_train, y_train)
et.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000934 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785


LGBMClassifier(random_state=1)

In [71]:
# Predicting and evaluating
models = {'Random Forest': rf, 'Extra Trees': et, 'XGBoost': xgb, 'LightGBM': lgbm}
for name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")


Random Forest Accuracy: 0.7913
Extra Trees Accuracy: 0.7672
XGBoost Accuracy: 0.7935
LightGBM Accuracy: 0.8034


In [79]:
# Define the best hyperparameters
best_params = {
    'n_estimators': 100,
    'max_depth': 5,
    'min_samples_split': 10,
    'min_samples_leaf': 10
}

best_clf = ExtraTreesClassifier(random_state=1, **best_params)

best_clf.fit(X_train, y_train)
y_pred_best = best_clf.predict(X_test)


accuracy_best = accuracy_score(y_test, y_pred_best)
print("Accuracy of the new optimal model:", accuracy_best)


Accuracy of the new optimal model: 0.8055358410220014
